# Data Gathering and Cleaning


## Pushshift API Wrapper

In [1]:
import pandas as pd
import time
import requests
import datetime as dt

In [7]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

def query_pushshift(subreddit, kind='submission', skip=5, times=50, 
                    subfield = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments',
                                'score', 'is_self']):
    
    # get the base url that contains information I want to scrape where 'kind' are all submitted posts
    # and 'subreddit' is the specified subreddit. Get 500 posts.
    stem = f"https://api.pushshift.io/reddit/search/{kind}/?subreddit={subreddit}&size=500"
    
    # instantiate list to contain 
    mylist = []
    
    # scrape posts from the subreddit 'times' times
    for x in range(1, times + 1):
        
        # Get posts 'skip' * 'x' days ago
        URL = f"{stem}&after={skip * x}d"
        print(URL)
        
        # Scrape URL
        response = requests.get(URL)
        
        # Give me an AssertionError if status code not 200
        assert response.status_code == 200
        
        # Of the HTML scraped, take the values of 'data'
        the_json=response.json()
 
        # turn the data into a dataframe
        df = pd.DataFrame.from_dict(the_json['data'])
        
        # append the dataframe to mylist
        mylist.append(df)
        
        # wait to not overrun Reddit's resources
        time.sleep(3)
        
    # concatenate the dataframes together as one large dataframe, full
    full = pd.concat(mylist, sort=False)
    if kind == "submission":
        
        # take all speficied data
        full = full[subfield]
        
        # drop duplicate rows
        full = full.drop_duplicates()
        full = full.loc[full['is_self'] == True]
          
    # date the the post was... posted
    _timestamp = full["created_utc"].apply(get_date)
    full['timestamp'] = _timestamp
    print(full.shape)
    return full

In [8]:
# Scrape environment data
df_env = query_pushshift(subreddit='environment', skip=5, times=50)

https://api.pushshift.io/reddit/search/submission/?subreddit=environment&size=500&after=5d
https://api.pushshift.io/reddit/search/submission/?subreddit=environment&size=500&after=10d
https://api.pushshift.io/reddit/search/submission/?subreddit=environment&size=500&after=15d
https://api.pushshift.io/reddit/search/submission/?subreddit=environment&size=500&after=20d
https://api.pushshift.io/reddit/search/submission/?subreddit=environment&size=500&after=25d
https://api.pushshift.io/reddit/search/submission/?subreddit=environment&size=500&after=30d
https://api.pushshift.io/reddit/search/submission/?subreddit=environment&size=500&after=35d
https://api.pushshift.io/reddit/search/submission/?subreddit=environment&size=500&after=40d
https://api.pushshift.io/reddit/search/submission/?subreddit=environment&size=500&after=45d
https://api.pushshift.io/reddit/search/submission/?subreddit=environment&size=500&after=50d
https://api.pushshift.io/reddit/search/submission/?subreddit=environment&size=500

In [9]:
# Scrape technology data
df_tech = query_pushshift(subreddit='technology', skip=5, times=50)

https://api.pushshift.io/reddit/search/submission/?subreddit=technology&size=500&after=5d
https://api.pushshift.io/reddit/search/submission/?subreddit=technology&size=500&after=10d
https://api.pushshift.io/reddit/search/submission/?subreddit=technology&size=500&after=15d
https://api.pushshift.io/reddit/search/submission/?subreddit=technology&size=500&after=20d
https://api.pushshift.io/reddit/search/submission/?subreddit=technology&size=500&after=25d
https://api.pushshift.io/reddit/search/submission/?subreddit=technology&size=500&after=30d
https://api.pushshift.io/reddit/search/submission/?subreddit=technology&size=500&after=35d
https://api.pushshift.io/reddit/search/submission/?subreddit=technology&size=500&after=40d
https://api.pushshift.io/reddit/search/submission/?subreddit=technology&size=500&after=45d
https://api.pushshift.io/reddit/search/submission/?subreddit=technology&size=500&after=50d
https://api.pushshift.io/reddit/search/submission/?subreddit=technology&size=500&after=55d


In [10]:
# Randomly drop rows in tech df to have balanced classes with env data
df_tech = df_tech.sample(n=2951)
df_tech.shape

(2951, 9)

In [11]:
# Confirm shapes are now the same
df_env.shape

(2951, 9)

## Combine and save to csv

In [12]:
df = pd.concat([df_env, df_tech], ignore_index=True)

In [13]:
df.shape

(5902, 9)

In [14]:
# save raw file
df.to_csv('../data/posts_raw.csv', index=False)

# Data Cleaning

- Address subtext issues 
- Combine selftext and titles into one column
- Drop 'AutoModerator' authored rows
- Drop unneeded columns
- remove non-letter characters

In [59]:
# Import for cleaning
posts = pd.read_csv('../data/posts_raw.csv')

In [60]:
posts['subreddit'].value_counts()

environment    2951
technology     2951
Name: subreddit, dtype: int64

In [61]:
# Check text in selftext column - need to change [removed] and [deleted]
posts['selftext'].value_counts()

[removed]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [62]:
posts[(posts['selftext']=='[removed]')]

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp
4,Breathing Pattern Disorders Caused by Environm...,[removed],environment,1579883335,ValiantDance,0,1,True,2020-01-24 11:28:55
15,My village don't recycle,[removed],environment,1579942566,xdbeizz,0,1,True,2020-01-25 03:56:06
27,No Deal for Nature campaign,[removed],environment,1580045119,total_green_future,0,1,True,2020-01-26 08:25:19
42,"If climate change is such a threat to the USA,...",[removed],environment,1580169070,ChopFinsOffOrcas,0,1,True,2020-01-27 18:51:10
63,Blockchain Technology in Sustainable Fashion -...,[removed],environment,1579457646,hvvzl,0,1,True,2020-01-19 13:14:06
...,...,...,...,...,...,...,...,...,...
5895,[Academic] Are we now seeing the benefits of J...,[removed],technology,1561319930,E-Ude,1,1,True,2019-06-23 15:58:50
5896,Why do my two different devices on the same ne...,[removed],technology,1565235871,SenseiKahi,2,1,True,2019-08-07 23:44:31
5897,The wAIting Game,[removed],technology,1561308177,Wenderu84,2,1,True,2019-06-23 12:42:57
5898,Photo Lab PRO Picture Editor v3.6.16,[removed],technology,1570894891,KISKASKEY,0,1,True,2019-10-12 11:41:31


In [63]:
# Fill empty self text values with empty string
posts['selftext'] = posts['selftext'].replace('[removed]',' ')

In [64]:
posts[['selftext']]

,selftext
0,https://www.dailymail.co.uk/news/article-7922...
1,There is a search engine called [Ecosia](https...
2,[Vandana Shiva](https://youtu.be/MNM833K22LM) ...
3,"If you have a weak stomach, I wouldn’t watch t..."
4,
...,...
5897,
5898,
5899,
5900,In my case\n\nI had to re-do a Video thus putt...


In [65]:
# Check for posts by the 'AutoModerator'
posts.drop(posts[(posts['author']=='AutoModerator')].index, inplace=True)

In [66]:
# Confirm none left
posts[(posts['author']=='AutoModerator')]

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp


In [67]:
# combine selftext and title columns into one text column
posts['text'] = posts['selftext'] + posts['title']

In [68]:
# confirm new column created
posts.head()

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp,text
0,Tesco to cut huge plastic use by removing wrap...,https://www.dailymail.co.uk/news/article-7922...,environment,1579839932,attooree,1,1,True,2020-01-23 23:25:32,https://www.dailymail.co.uk/news/article-7922...
1,The easiest ways to reforest Earth,There is a search engine called [Ecosia](https...,environment,1579854824,jedzoka,1,1,True,2020-01-24 03:33:44,There is a search engine called [Ecosia](https...
2,'Bill Gates is continuing the work of Monsanto...,[Vandana Shiva](https://youtu.be/MNM833K22LM) ...,environment,1579863401,CommonEmployment,2,1,True,2020-01-24 05:56:41,[Vandana Shiva](https://youtu.be/MNM833K22LM) ...
3,This is how the Chinese illegally turn toxic p...,"If you have a weak stomach, I wouldn’t watch t...",environment,1579882738,MBlaizze,0,1,True,2020-01-24 11:18:58,"If you have a weak stomach, I wouldn’t watch t..."
4,Breathing Pattern Disorders Caused by Environm...,,environment,1579883335,ValiantDance,0,1,True,2020-01-24 11:28:55,Breathing Pattern Disorders Caused by Environ...


In [69]:
# Drop all columns except subreddit identifier and text
posts.drop(columns=['title',
                    'selftext',
                    'created_utc',
                    'author',
                    'num_comments',
                    'score',
                    'is_self',
                   'timestamp'], inplace=True)

In [70]:
posts.head()

,subreddit,text
0,environment,https://www.dailymail.co.uk/news/article-7922...
1,environment,There is a search engine called [Ecosia](https...
2,environment,[Vandana Shiva](https://youtu.be/MNM833K22LM) ...
3,environment,"If you have a weak stomach, I wouldn’t watch t..."
4,environment,Breathing Pattern Disorders Caused by Environ...


In [71]:
# Convert environment and tech into binary labels
posts['subreddit'] = posts['subreddit'].map({'environment': 0, 'technology': 1})

In [72]:
posts['subreddit'].value_counts()

0    2951
1    2943
Name: subreddit, dtype: int64

In [75]:
# check for null values
posts.isnull().sum()

subreddit      0
text         377
dtype: int64

In [83]:
# drop rows with null values
posts.dropna(inplace=True)

In [84]:
posts.isnull().sum()

subreddit    0
text         0
dtype: int64

## Save to csv

In [85]:
posts.to_csv('../data/posts_clean.csv', index=False)